In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# ✅ ตั้งค่า WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,600")  # เปิดหน้าต่างขนาดเล็ก
driver = webdriver.Chrome(options=options)

# ✅ URL ของ Google Form (เปลี่ยนเป็นของคุณ)
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSepTZLYzLk_vGFsturC4HAR5pdKv3P7SNxxZN4weli_x0o0Fw/viewform"

# ✅ เปิดฟอร์มและรอโหลด
driver.get(form_url)
time.sleep(3)

# ✅ ฟังก์ชันสุ่มกรอกฟอร์ม
def fill_form():
    while True:
        try:
            all_filled = True  # เช็คว่ากรอกครบทุกคำถามหรือยัง

            # ✅ ดึงคำถามทั้งหมดในหน้านั้น
            questions = driver.find_elements(By.XPATH, "//div[contains(@class, 'freebirdFormviewerComponentsQuestionBaseRoot')]")
            for q in questions:
                try:
                    # ✅ Radio Button (ตัวเลือกเดียว)
                    radio_options = q.find_elements(By.XPATH, ".//label")
                    if radio_options:
                        choice = random.choice(radio_options)
                        driver.execute_script("arguments[0].scrollIntoView();", choice)  # เลื่อนให้มองเห็น
                        choice.click()
                        print(f"🎯 เลือก Radio Button: {choice.text}")
                        continue

                    # ✅ Checkbox (เลือกหลายตัว)
                    checkbox_options = q.find_elements(By.XPATH, ".//label")
                    if checkbox_options:
                        selected_checkboxes = random.sample(checkbox_options, random.randint(1, len(checkbox_options)))
                        for checkbox in selected_checkboxes:
                            driver.execute_script("arguments[0].scrollIntoView();", checkbox)  # เลื่อนให้มองเห็น
                            checkbox.click()
                        print(f"🟢 เลือก Checkbox {len(selected_checkboxes)} ตัว")
                        continue

                except Exception as e:
                    print(f"⚠️ พบข้อผิดพลาด: {str(e)}")

            # ✅ ตรวจสอบว่าทุกคำถามถูกตอบแล้วหรือยัง
            errors = driver.find_elements(By.XPATH, "//div[contains(text(), 'จำเป็นต้องตอบคำถามนี้')]")
            if errors:
                all_filled = False  # ยังมีคำถามที่ไม่ได้ตอบ

            # ✅ ถ้าตอบครบทุกข้อแล้ว ค่อยกดปุ่ม "ถัดไป"
            if all_filled:
                next_button = driver.find_elements(By.XPATH, "//span[text()='ถัดไป']")
                if next_button:
                    next_button[0].click()
                    time.sleep(2)  # รอโหลดหน้าใหม่
                else:
                    break  # ถ้าไม่มีปุ่ม "ถัดไป" แสดงว่าเป็นหน้าสุดท้าย → ออกจากลูป

        except:
            break  # ถ้าพบ Error ออกจากลูป

# ✅ กรอกฟอร์ม 10 ครั้ง
for i in range(10):
    print(f"📌 กำลังกรอกฟอร์มครั้งที่ {i+1}/10 ...")
    driver.get(form_url)
    time.sleep(2)
    fill_form()

    # ✅ ตรวจสอบว่ากรอกข้อมูลครบก่อนกดส่ง
    errors = driver.find_elements(By.XPATH, "//div[contains(text(), 'จำเป็นต้องตอบคำถามนี้')]")
    if errors:
        print("❌ พบข้อผิดพลาด: มีคำถามที่ยังไม่ได้ตอบ ฟอร์มจะไม่ถูกส่ง")
    else:
        # ✅ กดปุ่ม "ส่งฟอร์ม" (Submit) ในหน้าสุดท้าย
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='ส่ง']"))
        )
        submit_button.click()
        print(f"✅ ฟอร์มครั้งที่ {i+1} ถูกส่งแล้ว!")

    time.sleep(3)

# ✅ ปิด WebDriver
driver.quit()
print("🎉 กรอกฟอร์มเสร็จเรียบร้อย!")
